*This notebook contains exerpts from the [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; the content is available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).*
 
 *The text is released under the [CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode), and code is released under the [MIT license](https://opensource.org/licenses/MIT). If you find this content useful, please consider supporting the work by [buying the book](http://shop.oreilly.com/product/0636920034919.do)!*

# Introduction to NumPy

Datasets can come from a wide range of sources and a wide range of formats, including be collections of documents, collections of images, collections of sound clips, collections of numerical measurements, or nearly anything else. Despite this apparent heterogeneity, it will help us to think of all data fundamentally as arrays of numbers.

For example, images–particularly digital images–can be thought of as simply two-dimensional arrays of numbers representing pixel brightness across the area. Sound clips can be thought of as one-dimensional arrays of intensity versus time. Text can be converted in various ways into numerical representations, perhaps binary digits representing the frequency of certain words or pairs of words. No matter what the data are, the first step in making it analyzable will be to transform them into arrays of numbers. For this reason, efficient storage and manipulation of numerical arrays is absolutely fundamental to the process of doing data science.

NumPy (short for Numerical Python) provides an efficient interface to store and operate on dense data buffers. In some ways, NumPy arrays are like Python's built-in list type, but NumPy arrays provide much more efficient storage and data operations as the arrays grow larger in size. NumPy arrays form the core of nearly the entire ecosystem of data science tools in Python, so time spent learning to use NumPy effectively will be valuable no matter what aspect of data science interests you.

# Understanding data types in Python

Effective data-driven science and computation requires understanding how data is stored and manipulated. This section outlines and contrasts how arrays of data are handled in the Python language itself, and how NumPy improves on this. Understanding this difference is fundamental to understanding much of the material throughout the rest of the book.

Users of Python are often drawn-in by its ease of use, one piece of which is dynamic typing. While a statically-typed language like C or Java requires each variable to be explicitly declared, a dynamically-typed language like Python skips this specification. This means, for example, that we can assign any kind of data to any variable.

This sort of flexibility is one piece that makes Python and other dynamically-typed languages convenient and easy to use. Understanding how this works is an important piece of learning to analyze data efficiently and effectively with Python. But what this type-flexibility also points to is the fact that Python variables are more than just their value; they also contain extra information about the type of the value.

## A Python integer is more than just an integer

The standard Python implementation is written in C. This means that every Python object is simply a cleverly-disguised C structure, which contains not only its value, but other information as well. For example, when we define an integer in Python, such as `x = 10000`, `x` is not just a "raw" integer. It's actually a pointer to a compound C structure, which contains several values (Python 3.4):

- `ob_refcnt`, a reference count that helps Python silently handle memory allocation and deallocation
- `ob_type`, which encodes the type of the variable
- `ob_size`, which specifies the size of the following data members
- `ob_digit`, which contains the actual integer value that we expect the Python variable to represent.

This means that there is some overhead in storing an integer in Python as compared to an integer in a compiled language like C, as illustrated in the following figure:

![cint](images/cint_vs_pyint.png)

Here `PyObject_HEAD` is the part of the structure containing the reference count, type code, and other pieces mentioned before.

Notice the difference here: a C integer is essentially a label for a position in memory whose bytes encode an integer value. A Python integer is a pointer to a position in memory containing all the Python object information, including the bytes that contain the integer value. This extra information in the Python integer structure is what allows Python to be coded so freely and dynamically. All this additional information in Python types comes at a cost, however, which becomes especially apparent in structures that combine many of these objects.

## A Python list is more than just a list

Let's consider now what happens when we use a Python data structure that holds many Python objects. The standard mutable multi-element container in Python is the list. We can create a list of integers as follows:

In [3]:
L = list(range(10))
L

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [4]:
type(L[0])

int

Or, similarly, a list of strings:

In [5]:
L2 = [str(c) for c in L]
L2

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [6]:
type(L2[0])

str

Because of Python's dynamic typing, we can also create heterogeneous lists:

In [15]:
L3 = [True, "2", 3.0, 4]
[type(item) for item in L3]

[bool, str, float, int]

But this flexibility comes at a cost: to allow these flexible types, each item in the list must contain its own type info, reference count, and other information–that is, each item is a complete Python object. In the special case that all variables are of the same type, much of this information is redundant: it can be much more efficient to store data in a fixed-type array. The difference between a dynamic-type list and a fixed-type (NumPy-style) array is illustrated in the following figure:

![figure](images/array_vs_list.png)

At the implementation level, the array essentially contains a single pointer to one contiguous block of data. The Python list, on the other hand, contains a pointer to a block of pointers, each of which in turn points to a full Python object like the Python integer we saw earlier. Again, the advantage of the list is flexibility: because each list element is a full structure containing both data and type information, the list can be filled with data of any desired type. Fixed-type NumPy-style arrays lack this flexibility, but are much more efficient for storing and manipulating data.

## Fixed-type arrays in Python
Python offers several different options for storing data in efficient, fixed-type data buffers. The built-in `array` module can be used to create dense arrays of a uniform type:

In [18]:
import array
L = list(range(10))
A = array.array('i', L)
A

array('i', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

Here, `i` is a type code indicating the contents are integers.

Much more useful, however, is the `ndarray` object of the NumPy package. While Python's `array` object provides efficient storage of array-based data, NumPy adds to this efficient *operations* on that data.

## Creating arrays from Python lists

In [19]:
import numpy as np

In [20]:
# Integer array
np.array([1, 4, 2, 5, 3])

array([1, 4, 2, 5, 3])

In [46]:
# NumPy is constrained to arrays that all contain the same type
# and will otherwise upcast if possible.
# Here, integers are up-cast to floating point
np.array([3.14, 4, 2, 3])

array([3.14, 4.  , 2.  , 3.  ])

In [29]:
# If we want to explicitly set the data type of the resulting array,
# we can use to `dtype` keyword:
np.array([1, 2, 3, 4], dtype='float32')

array([1., 2., 3., 4.], dtype=float32)

In [30]:
# NumPy arrays can explicitly be multi-dimensional
# Nested lists result in multi-dimensional arrays
# The inner lists are treated as rows of the resulting 2D array
np.array([range(i, i + 3) for i in [2, 4, 6]])

array([[2, 3, 4],
       [4, 5, 6],
       [6, 7, 8]])

## Creating arrays from scratch
Especially for larger arrays, it is more efficient to create arrays from scratch using routines built into NumPy.

In [31]:
# Create a length-10 integer array filled with zeros
np.zeros(10, dtype=int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [32]:
# Create a 3x5 floating-point array filled with ones
np.ones((3, 5), dtype=float)

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

In [34]:
# Create a 3x5 array filled with 3.14
np.full((3, 5), 3.14)

array([[3.14, 3.14, 3.14, 3.14, 3.14],
       [3.14, 3.14, 3.14, 3.14, 3.14],
       [3.14, 3.14, 3.14, 3.14, 3.14]])

In [35]:
# Create an array filled with a linear sequence
# Starting at 0, ending at 20, stepping by 2
# Similar to the built-in range() function
np.arange(0, 20, 2)

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18])

In [36]:
# Create an array of five values evenly spaced between 0 and 1
np.linspace(0, 1, 5)

array([0.  , 0.25, 0.5 , 0.75, 1.  ])

In [38]:
# Create a 3x3 array of uniformly distributed random values
# with mean 0 and standard deviation 1
np.random.normal(0, 1, (3, 3))

array([[-0.50369429,  1.72535291, -2.30143946],
       [-0.36513912,  0.51349902, -1.73414152],
       [ 0.57179508, -1.74969317,  1.55324075]])

In [39]:
# Create a 3x3 array of random integers in the interval [0, 10)
np.random.randint(0, 10, (3, 3))

array([[5, 8, 8],
       [2, 0, 3],
       [2, 4, 4]])

In [40]:
# Create a 3x3 identity matrix
np.eye(3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [41]:
# Create an uninitialized array of three integers
# The values will be whatever happens to already exist at that memory location
np.empty(3)

array([1., 1., 1.])

## NumPy standard data types
NumPy arrays contain values of a single type, so it is important to have detailed knowledge of those types and their limitations. Because NumPy is built in C, the types will be familiar to users of C, Fortran, and other related languages.

The standard NumPy data types are listed in the following table. Note that when constructing an array, they can be specified using a string or using the associated NumPy object:

In [47]:
np.zeros(10, dtype='int16')

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int16)

In [48]:
np.zeros(10, dtype=np.int16)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int16)

 | Data type	    | Description |
 |---------------|-------------|
 | ``bool_``     | Boolean (True or False) stored as a byte |
 | ``int_``      | Default integer type (same as C ``long``; normally either ``int64`` or ``int32``)| 
 | ``intc``      | Identical to C ``int`` (normally ``int32`` or ``int64``)| 
 | ``intp``      | Integer used for indexing (same as C ``ssize_t``; normally either ``int32`` or ``int64``)| 
 | ``int8``      | Byte (-128 to 127)| 
 | ``int16``     | Integer (-32768 to 32767)|
 | ``int32``     | Integer (-2147483648 to 2147483647)|
 | ``int64``     | Integer (-9223372036854775808 to 9223372036854775807)| 
 | ``uint8``     | Unsigned integer (0 to 255)| 
 | ``uint16``    | Unsigned integer (0 to 65535)| 
 | ``uint32``    | Unsigned integer (0 to 4294967295)| 
 | ``uint64``    | Unsigned integer (0 to 18446744073709551615)| 
 | ``float_``    | Shorthand for ``float64``.| 
 | ``float16``   | Half precision float: sign bit, 5 bits exponent, 10 bits mantissa| 
 | ``float32``   | Single precision float: sign bit, 8 bits exponent, 23 bits mantissa| 
 | ``float64``   | Double precision float: sign bit, 11 bits exponent, 52 bits mantissa| 
 | ``complex_``  | Shorthand for ``complex128``.| 
 | ``complex64`` | Complex number, represented by two 32-bit floats| 
 | ``complex128``| Complex number, represented by two 64-bit floats| 